In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Combiner les colonnes 'preproc_title' et 'preproc_body' en une seule colonne 'combined_text'
df['combined_text'] = df['preproc_title'].apply(' '.join) + " " + df['preproc_body'].apply(' '.join)

# 2. Entraîner le modèle Word2Vec sur le corpus combiné
corpus = df['combined_text'].apply(lambda x: x.split())
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec_model.model")

# 3. Fonction pour obtenir l'embedding d'une phrase
def get_sentence_embedding(sentence, model):
    embeddings = [model.wv[word] for word in sentence if word in model.wv]
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)
    
# Appliquer la fonction à tout le DataFrame pour obtenir un embedding pour chaque texte
df['embedding'] = df['combined_text'].apply(lambda x: get_sentence_embedding(x.split(), word2vec_model))

# 4. Préparer les étiquettes (tags) avec MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['preproc_tags'])

# 5. Préparer les données d'entraînement et de test
X = df['embedding'].tolist()  # Convertir la colonne embedding en une liste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Entraîner la régression logistique CETTE ETAPE EST LONGUE
logistic_model = OneVsRestClassifier(LogisticRegression(C=0.1, max_iter=200, solver='saga', random_state=42))
logistic_model.fit(X_train, y_train)

In [ ]:
# 7. Faire des prédictions
y_pred = logistic_model.predict(X_test)

# 8. Évaluer les performances du modèle
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

KeyboardInterrupt: 